# MANIPULATION DES DONNEES

*Source: cette série d'exercices s'inpire très largement des tutoriels de Simon Ejdemyr: https://sejdemyr.github.io/r-tutorials/statistics/*

On commence par ouvrir les données du TD1. 

In [ ]:
world<-read.csv("TD1_world_small.csv")
head(world)
summary(world)

# 1. Sous échantillon

Il y a plusieurs moyens de se concentrer sur un sous-échantillon sur R. Par exemple, imaginons que nous souhaitions travailler sur le sous-échantillon de la région 'Afrique'. 

In [ ]:
afr1 <- world[world$region == "Africa", ]   #option 1: crochets 
dim(afr1)
afr1 <- subset(world, region == "Africa")   #option 2: commande subset()
dim(afr1)
library(dplyr) 
afr1 <- filter(world, region == "Africa")   #option 3: commande filter() du package dplyr
dim(afr1)
head(afr1)

On peut créer un sous échantillon encore plus restreint, les pays qui sont africains et dont le score 'polity' est de 15 au moins: 

In [ ]:
afr2 <- world[world$region == "Africa" & world$polityIV >= 15, ]          #option 1
afr2 <- subset(world, region == "Africa" & polityIV >= 15)                #option 2
afr2 <- filter(world, region == "Africa", polityIV >= 15)                 #option 3
head(afr2)

*NOTE: nous utilisons ici des opérateurs logiques: ==, &, >=. Il y a un grande différence entre = et ==.*

- *= fonctionne comme l'opérateur de définition <-*

- *a==3 teste si a est égal à 3 et dit si l'égalité est vraie ou fausse. *

# 2. Nouvelles variables

La méthode la plus commune est d'utiliser l'opérateur **$** suivi de la nouvelle variable :

In [ ]:
world$gdp_log <- log(world$gdppcap08) 
#cette commande crée le log du GDP et l'ajoute au dataframe

In [ ]:
world$democ <- ifelse(world$polity>10, 1, 0)
#cette commande crée la variable dummy = 1 si polity eest supérieure à 10, 0 sinon
#dans la parenthèse, on donne :
    #d'abord le test qui peut être vrai ou faux, 
    #ensuite la valeur que la variable doit prendre si l'argument est vrai,
    #enfin la valeur si l'argument est faux.

In [ ]:
head(world)

NOTE: On peut faire la même chose en une seule et unique commande avec **mutate()** de **dplyr** :

In [ ]:
world <- mutate(world, gdp_log = log(gdppcap08), 
                       democ = ifelse(polityIV > 10, 1, 0))
head(world)

# 3. Les variables facteur

Dans la base, la variable *region* est une variable facteur:

In [ ]:
class(world$region)

Les variables facteur sont importantes, ce sont des variables catégorielles. Voici les catégories prises par region (appelées 'levels' dans R): 

In [ ]:
levels(world$region)

Le fait que *region* soit de type facteur et non de type caractère est important pour faciliter le recodage et les estimations. Pour transformer une variable qui n'est pas un facteur en facteur, on utilise **factor()**. 

Si nous souhaitons regrouper les régions européennes dans une même catégorie, nous pouvons créer facilement une nouvelle variable facteur: 

In [ ]:
world$largeregion<-world$region
levels(world$largeregion)<-c("Africa", "Asia-Pacific", "Europe", "Middle East", "N. America", "S. America", "Europe", "Europe") 

In [ ]:
table(world$region)

In [ ]:
table(world$largeregion)

# 4. Ordonner les données

Le moyen le plus simple d'ordonner les données est d'utiliser **arrange()** du package **dplyr**. 

In [ ]:
head(world)

In [ ]:
world<-arrange(world,gdp_log)
head(world)

In [ ]:
world<-arrange(world, desc(gdp_log))
head(world)

In [ ]:
world<-arrange(world, region, country)
head(world)

# 5. Combiner les commandes avec "piping" (**%>%**)

Le package dplyr rend possible l'écriture de commandes compactes en combinant plusieurs types d'opérations, gâce à l'opérateur **%>%** qui permet d'enchaîner les commandes dans l'ordre indiqué.

Par exemple, nous pouvons souhaiter 
- sélectionner le sous échantillon des pays sud américains avec un score polity en dessous de 20, 
- créer deux nouvelles variables - gdp_log et democ, 
- garder seulement certaines variables - country, gdp_log et democ, 
- et réordonner les variables en se basant sur le log gdp avec un ordre décroissant. 

Avec le piping de dplyr, cela donne: 

In [ ]:
newdata <- world %>%
            filter(region == "S. America", polityIV<20) %>%
            mutate(gdp_log = log(gdppcap08),
                   democ = ifelse(polityIV>10,1,0)) %>%
            select(country,gdp_log,democ) %>%
            arrange(desc(gdp_log))
head(newdata)
summary(newdata)
newdata

# Exercice

Ouvrez à nouveau les données TD1_world_small.csv.

Créez 2 variables dans la base world : 
- une variable qui recode polityIV (qui varie de 0 à 20) en une variabel qui varie de -10 à 10, que l'on nommera polityIV2. 
- une variable catégorielle qui classe les pays comme 'riches' et 'pauvres' selon un seuil du gdppcap08 que vous jugerez pertinent.

Créez un sous-échantillon des pays européens. 

Supprimez la variable *region* du sous-échantillon.

Ordonnez les données du sous-échantillon sur la base de la variable *polityIV2*.

Ouvrez à nouveau les données. Refaites les questions ci-dessus en utilisant la fonctionnalité *piping*. 

Combien de pays européens sont riches selon votre encodage ? Combien sont pauvres ? Quel est le poucentage de pays européens qui ont un score polityIV de 18 et plus ?